In [10]:
import numpy as np
import pandas as pd

# Ordinary Least Square Ridge Regression

In [11]:
class OLSRidge:
    def __init__(self,alpha=0.1):
        self.coef=None
        self.intercept=None
        self.alpha=alpha

    def fit(self,X_train,y_train):
        X_train=np.insert(X_train,0,1,axis=1)
        I=np.identity(X_train.shape[1])
        I[0][0]=0
        beta=np.linalg.inv(np.dot(X_train.T,X_train)+self.alpha*I).dot(X_train.T).dot(y_train)
        self.intercept=beta[0]
        self.coef=beta[1:]

    def predict(self,X_test):
        return np.dot(X_test,self.coef)+self.intercept
        
        

# Gradient Descent Ridge Regression

In [17]:
class GDRidge:
    def __init__(self,epochs,learning_rate,alpha):
        self.learning_rate=learning_rate
        self.epochs=epochs
        self.alpha=alpha
        self.intercept=None
        self.coef=None

    def fit(self,X_train,y_train):
        self.coef=np.ones(X_train.shape[1])
        self.intercept=0
        thetha=np.insert(self.coef,0,self.intercept)
        X_train=np.insert(X_train,0,1,axis=1)

        for i in range(self.epochs):
            thetha_der=np.dot(X_train.T,X_train).dot(thetha)-np.dot(X_train.T,y_train)+self.alpha*thetha
            thetha=thetha-self.learning_rate*thetha_der
        self.coef=thetha[1:]
        self.intercept=thetha[0]
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef)+self.intercept

        

# testing OLSRidge

In [13]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error



#generate sample regression data
X, y = make_regression(n_samples=1000, n_features=7, noise=50, random_state=42)

#Split into train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Custom Ridge Regression
my_ridge = OLSRidge(alpha=1.0)
my_ridge.fit(X_train, y_train)
y_pred_custom = my_ridge.predict(X_test)

#Sklearn Ridge Regression
sk_ridge = Ridge(alpha=1.0, fit_intercept=True)
sk_ridge.fit(X_train, y_train)
y_pred_sklearn = sk_ridge.predict(X_test)

#Compare
print("Custom Ridge Coefficients:", my_ridge.coef)
print("Sklearn Ridge Coefficients:", sk_ridge.coef_)

print("Custom Ridge Intercept:", my_ridge.intercept)
print("Sklearn Ridge Intercept:", sk_ridge.intercept_)

print("Custom Ridge MSE:", mean_squared_error(y_test, y_pred_custom))
print("Sklearn Ridge MSE:", mean_squared_error(y_test, y_pred_sklearn))


Custom Ridge Coefficients: [60.78942109 93.13790238  8.74915074 -1.06748493 33.97542321 34.10982761
 15.61194872]
Sklearn Ridge Coefficients: [60.78942109 93.13790238  8.74915074 -1.06748493 33.97542321 34.10982761
 15.61194872]
Custom Ridge Intercept: 1.3582426049486596
Sklearn Ridge Intercept: 1.3582426049486593
Custom Ridge MSE: 3000.5284047513373
Sklearn Ridge MSE: 3000.528404751338


# testing GDRidge

In [23]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score


# Generate synthetic regression data
X, y = make_regression(n_samples=500, n_features=5, noise=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train custom GDRidge
gd_model = GDRidge(epochs=1000, learning_rate=0.001, alpha=0.1)
gd_model.fit(X_train, y_train)
gd_preds = gd_model.predict(X_test)

# Train sklearn Ridge using gradient-based solver
sk_model = Ridge(alpha=0.1, solver='sag', max_iter=1000, tol=1e-3, random_state=42)
sk_model.fit(X_train, y_train)
sk_preds = sk_model.predict(X_test)

# Compare
print("Custom GDRidge Coefficients:\n", gd_model.coef)
print("Sklearn Ridge Coefficients:\n", sk_model.coef_)

print("\nCustom GDRidge Intercept:", gd_model.intercept)
print("Sklearn Ridge Intercept:", sk_model.intercept_)

print("\nCustom GDRidge R2 Score:", r2_score(y_test, gd_preds))
print("Sklearn Ridge R2 Score:", r2_score(y_test, sk_preds))

# Optional: Check difference
print("\nMax Coefficient Difference:", np.abs(gd_model.coef - sk_model.coef_).max())
print("Intercept Difference:", abs(gd_model.intercept - sk_model.intercept_))


Custom GDRidge Coefficients:
 [28.75197256 81.75205781 31.0283662  68.69718435 10.94001164]
Sklearn Ridge Coefficients:
 [28.64988964 81.71018658 31.08572067 68.77988248 10.90070253]

Custom GDRidge Intercept: 0.0459569392686471
Sklearn Ridge Intercept: 0.04419975530348497

Custom GDRidge R2 Score: 0.9910623807408622
Sklearn Ridge R2 Score: 0.9910736268592321

Max Coefficient Difference: 0.1020829201107567
Intercept Difference: 0.0017571839651621332
